In [234]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [235]:
df = pd.read_csv('yield_df.csv')

In [236]:
df.head()

,Unnamed: 0,area,item,year,yield,avg_rain,fertilizer,avg_temp
0,0,Albania,Maize,1990,36613,1485.0,121.0,16.37
1,1,Albania,Potatoes,1990,66667,1485.0,121.0,16.37
2,2,Albania,"Rice, paddy",1990,23333,1485.0,121.0,16.37
3,3,Albania,Sorghum,1990,12500,1485.0,121.0,16.37
4,4,Albania,Soybeans,1990,7000,1485.0,121.0,16.37


In [237]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [238]:
df.shape

(28242, 7)

In [239]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28242 entries, 0 to 28241
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   area        28242 non-null  object 
 1   item        28242 non-null  object 
 2   year        28242 non-null  int64  
 3   yield       28242 non-null  int64  
 4   avg_rain    28242 non-null  float64
 5   fertilizer  28242 non-null  float64
 6   avg_temp    28242 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 1.5+ MB


In [240]:
df.isnull().sum()

area          0
item          0
year          0
yield         0
avg_rain      0
fertilizer    0
avg_temp      0
dtype: int64

In [241]:
df.duplicated().sum()

2310

In [242]:
df.drop_duplicates(inplace=True)

In [243]:
df.duplicated().sum()

0

# Average rainfall check

In [244]:
def isStr(obj):
    try:
        float(obj)
        return False
    except:
        return True
drop_rows = df[df['avg_rain'].apply(isStr)].index

In [245]:
df = df.drop(drop_rows)

In [246]:
df['avg_rain'] = df['avg_rain'].astype(np.float64)

In [247]:
df['yield'].sum()

1996196943

# Train-Test split

In [248]:
col = ['year', 'avg_rain','fertilizer', 'avg_temp', 'area', 'item', 'yield']
df = df[col]
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [249]:
df.head(3)

,year,avg_rain,fertilizer,avg_temp,area,item,yield
0,1990,1485.0,121.0,16.37,Albania,Maize,36613
1,1990,1485.0,121.0,16.37,Albania,Potatoes,66667
2,1990,1485.0,121.0,16.37,Albania,"Rice, paddy",23333


In [250]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0, shuffle=True)

# Transformation

In [251]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
ohe = OneHotEncoder(drop='first')
scale = StandardScaler()

preprocesser = ColumnTransformer(
        transformers = [
            ('StandardScale', scale, [0, 1, 2, 3]),
            ('OHE', ohe, [4, 5]),
        ],
        remainder='passthrough'
)

In [252]:
X_train_dummy = preprocesser.fit_transform(X_train)
X_test_dummy = preprocesser.transform(X_test)

In [253]:
preprocesser.get_feature_names_out(col[:-1])

array(['StandardScale__year', 'StandardScale__avg_rain',
       'StandardScale__fertilizer', 'StandardScale__avg_temp',
       'OHE__area_Algeria', 'OHE__area_Angola', 'OHE__area_Argentina',
       'OHE__area_Armenia', 'OHE__area_Australia', 'OHE__area_Austria',
       'OHE__area_Azerbaijan', 'OHE__area_Bahamas', 'OHE__area_Bahrain',
       'OHE__area_Bangladesh', 'OHE__area_Belarus', 'OHE__area_Belgium',
       'OHE__area_Botswana', 'OHE__area_Brazil', 'OHE__area_Bulgaria',
       'OHE__area_Burkina Faso', 'OHE__area_Burundi',
       'OHE__area_Cameroon', 'OHE__area_Canada',
       'OHE__area_Central African Republic', 'OHE__area_Chile',
       'OHE__area_Colombia', 'OHE__area_Croatia', 'OHE__area_Denmark',
       'OHE__area_Dominican Republic', 'OHE__area_Ecuador',
       'OHE__area_Egypt', 'OHE__area_El Salvador', 'OHE__area_Eritrea',
       'OHE__area_Estonia', 'OHE__area_Finland', 'OHE__area_France',
       'OHE__area_Germany', 'OHE__area_Ghana', 'OHE__area_Greece',
       'OHE__a

# Model Training

In [254]:
#linear regression
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error


models = {
    'lr':LinearRegression(),
    'lss':Lasso(),
    'Rid':Ridge(),
    'Dtr':DecisionTreeRegressor(),
    'knn':KNeighborsRegressor(),
    'Rdf':RandomForestRegressor()
}
for name, md in models.items():
    md.fit(X_train_dummy,y_train)
    y_pred = md.predict(X_test_dummy)
    
    print(f"{name} : mae : {mean_absolute_error(y_test,y_pred)} score : {r2_score(y_test,y_pred)} mse: {mean_squared_error(y_test, y_pred)}")

lr : mae : 29907.447350409722 score : 0.7473125540079992 mse: 1817342943.5139735


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 67280771830.0, tolerance: 14848622817.505169
  model = cd_fast.sparse_enet_coordinate_descent(


lss : mae : 29893.99762450549 score : 0.7473261756207235 mse: 1817244976.0757167
Rid : mae : 29864.85302468091 score : 0.7473042393527811 mse: 1817402743.0023456
Dtr : mae : 3914.620397146713 score : 0.9797476817086801 mse: 145655861.89705032
knn : mae : 4616.9047619047615 score : 0.9849925066819024 mse: 107934772.83529592
Rdf : mae : 3701.9318353576245 score : 0.9877742255207816 mse: 87928487.66813745


# Model Selection

In [255]:
Rdf = RandomForestRegressor()
Rdf.fit(X_train_dummy,y_train)
Rdf.predict(X_test_dummy)

array([38148.69, 24380.93, 21049.82, ..., 21206.11, 34821.  , 89315.46])

# Predictive System

In [256]:
def prediction(Year, avg_rain, fertlizer, avg_temp, Area, Item):
    features = np.array([[Year, avg_rain, fertlizer, avg_temp, Area, Item]], dtype=object)
    transformed_features = preprocesser.transform(features)
    predicted_yield = Rdf.predict(transformed_features).reshape(1, -1)

    return predicted_yield[0]

Year = 2005
avg_rain = 1180.0
fertlizer = 12.00
avg_temp = 31.2
Area = 'India'
Item = 'Soybeans'
result = prediction(Year, avg_rain, fertlizer, avg_temp, Area, Item)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [257]:
result

array([14783.54])

# Pickle Files

In [258]:
import pickle
pickle.dump(Rdf,open('rdf.pkl','wb'))
pickle.dump(preprocesser,open('preprocessor.pkl','wb'))